# Thief Detector

Author: Mohamed Oussama NAJI

Date: March 27, 2024

## Table of Contents
1. [Introduction](#introduction)
2. [Implementation](#implementation)
    - [Get Live Video Feed from Webcam](#video-feed)
    - [Read First Frame and Store as Reference Background Image](#first-frame)
    - [Compute Absolute Difference between Current and First Frame](#frame-difference)
    - [Apply Threshold](#threshold)
    - [Find Contours](#contours)
    - [Check Contour Area and Draw Rectangle](#contour-area)
    - [Display Images](#display-images)
    - [Release Objects](#release-objects)
    - [Main Function](#main-function)
3. [Results](#results)
4. [Conclusion](#conclusion)

## Introduction <a id="introduction"></a>

This project demonstrates how to build a motion detection algorithm that alerts you when an unwanted visitor is detected in your home. The algorithm uses image processing techniques to detect changes in the monitored scene and triggers an alarm if an intruder is detected.

The main steps involved in the thief detector algorithm are:
1. Get the live video feed from your webcam.
2. Fix a scene (the place you want to monitor) and store it as a reference background image.
3. For every frame, check if there is any unwanted object inside the scene you are monitoring using background subtraction.
4. Output the text "UNSAFE" in red color on the top right of the frame when there is an intruder in the scene.
5. Save the live feed.

## Implementation <a id="implementation"></a>

### Get Live Video Feed from Webcam <a id="video-feed"></a>


In [ ]:
import cv2
import time

def get_video_feed(camera_index=0):
    cap = cv2.VideoCapture(camera_index)
    return cap

### Read First Frame and Store as Reference Background Image <a id="first-frame"></a>


In [ ]:
def initialize_background(cap):
    ret, reference_frame = cap.read()
    if not ret:
        print("Error: Unable to read the first frame.")
        exit()

    reference_frame = cv2.cvtColor(reference_frame, cv2.COLOR_BGR2GRAY)
    return reference_frame

### Compute Absolute Difference between Current and First Frame <a id="frame-difference"></a>


In [ ]:
def compute_difference(frame, reference_frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_diff = cv2.absdiff(gray, reference_frame)
    return frame_diff

### Apply Threshold <a id="threshold"></a>

In [ ]:
def apply_threshold(frame_diff, threshold_value=25):
    _, thresh = cv2.threshold(frame_diff, threshold_value, 255, cv2.THRESH_BINARY)
    return thresh

### Find Contours <a id="contours"></a>

In [ ]:
def find_contours(thresh):
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

### Check Contour Area and Draw Rectangle <a id="contour-area"></a>

In [ ]:
def check_contour_area(frame, contours, threshold_area=100):
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > threshold_area:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(frame, "UNSAFE", (frame.shape[1] - 150, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            print("Motion detected! Triggering alarm.")

### Display Images <a id="display-images"></a>

In [ ]:
def display_images(frame, thresh):
    cv2.imshow('Frame', frame)
    cv2.imshow('Motion', thresh)

### Release Objects <a id="release-objects"></a>

In [ ]:
def release_objects(cap):
    cap.release()
    cv2.destroyAllWindows()

### Update Reference Frame

In [ ]:
REFERENCE_UPDATE_INTERVAL = 5  # this is to update reference frame every 5s
def update_reference_frame(cap, reference_frame):
    ret, new_reference_frame = cap.read()
    if ret:
        new_reference_frame = cv2.cvtColor(new_reference_frame, cv2.COLOR_BGR2GRAY)
        return new_reference_frame
    else:
        return reference_frame

### Main Function <a id="main-function"></a>

In [ ]:
VIDEO_OUTPUT_FILE = 'thiefdetector2.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(VIDEO_OUTPUT_FILE, fourcc, 20.0, (640, 480))

def main():
    cap = get_video_feed()
    reference_frame = initialize_background(cap)
    last_reference_update = time.time()

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Unable to read a frame.")
            break

        frame_diff = compute_difference(frame, reference_frame)
        thresh = apply_threshold(frame_diff)
        contours = find_contours(thresh)
        check_contour_area(frame, contours)
        display_images(frame, thresh)
        out.write(frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            out.release()
            break

        if time.time() - last_reference_update > REFERENCE_UPDATE_INTERVAL:
            reference_frame = update_reference_frame(cap, reference_frame)
            last_reference_update = time.time()

    release_objects(cap)

if __name__ == "__main__":
    main()

## Results <a id="results"></a>

The thief detector algorithm was successfully implemented and tested using a webcam. The algorithm was able to detect motion in the monitored scene and trigger an alarm when an intruder was detected.

The output video file "thiefdetector2.mp4" was generated, which captures the live feed along with the "UNSAFE" text displayed in red color when motion is detected.


## Conclusion <a id="conclusion"></a>

In this project, we developed a thief detector using image processing techniques. The algorithm monitors a fixed scene and detects any unwanted objects or motion within that scene. When an intruder is detected, the algorithm triggers an alarm and displays the text "UNSAFE" in red color on the top right of the frame.

The thief detector can be further enhanced by incorporating additional features such as:
- Sending alerts or notifications to the user when motion is detected.
- Integrating with a security system to automatically trigger alarms or lock doors.
- Improving the accuracy of motion detection by using more advanced techniques like background modeling or deep learning-based object detection.

Overall, the thief detector provides a basic framework for building a motion detection system that can be used for home security or surveillance purposes.